In [1]:
import numpy as np
from scipy.special import digamma
from textblob import TextBlob
from collections import defaultdict, Counter

In [3]:
class VanillaLDA:
    """
    Class to apply LDA to toy corpus to make sense of LDA mechanisms.
    
    Assumptions on toy corpus : Size of the corpus is small enough so that 
     1. counting the exact number of vocabularies in corpus is easy
     2. corpus can be expressed in dense array
    """
    
    
    def __init__(self, n_docs, n_vocabs, n_topics, maxiter=500, tol=1e-4):
        """
        lambda, gamma are array of variational parameters of topic, topic proportion respectively.
         - lam : K * V array whose row refers to k-th topic
         - gam : D * K array whose row refers to topic proportion of d-th document
        """
        self.D = n_docs
        self.V = n_vocabs
        self.K = n_topics
        self.lam = np.ones((n_topics, n_vocabs))
        self.gam = np.ones((n_docs, n_topics))
        self.maxiter = maxiter
        self.tol = tol
    
    
    def fit(self, corpus, alpha=1., eta=1.):
        """
        Apply LDA to toy corpus.
         - corpus : 2-dim np.array of coordinates which refer to the count of vocabulary
         - alpha, eta : hyperparameters related to topic proportion and topic respectively
         
        Updating process iterates until Frobenius norm between both new and old lambda, gamma converges.
        As size of the corpus is assumed to be small enough, presented operation is really coarse.
        """
        phi = np.zeros((self.K, self.D, self.V))
        
        for iteration in range(self.maxiter):
            
            lam_old = self.lam.copy()
            gam_old = self.gam.copy()

            # phi
            digamma_lam = (digamma(self.lam).T - digamma(np.apply_along_axis(np.sum, 1, self.lam))).T
            digamma_gam = (digamma(self.gam).T - digamma(np.apply_along_axis(np.sum, 1, self.gam))).T
            for d in range(self.D):
                for v in range(self.V):
                    ln_phi_dv = digamma_gam[:,d] + digamma_lam[:,v]
                    phi[:,d,v] = np.exp(ln_phi_dv) / np.sum(np.exp(ln_phi_dv))
            
            # gamma
            for d in range(self.D):
                self.gam[d,:] = np.apply_along_axis(np.sum, 1, phi[:,d,:].T * corpus[d,:])
            
            # lambda
            for k in range(self.K):
                self.lam[k,:] = np.apply_along_axis(np.sum, 1, corpus * phi[k,:,:])
                
            # Convergence
            bool1 = np.linalg.norm(lam_old - self.lam) < self.tol
            bool2 = np.linalg.norm(gam_old - self.gam) < self.tol
            if bool1 and bool2:
                break
                
        if (iteration+1) == self.maxiter:
            print("Algorithm did not converge")

# Data generating process

First of all, generate toy data to test the performance of the implemented algorithm. For simplicity, collect nouns from 3 distinct wikipedia articles. Title of each article was
1. Bayes' theorem

In [4]:
bayes = [
    "In probability theory and statistics, Bayes's theorem describes the probability of an event, based on prior knowledge of conditions that might be related to the event.",
    "For example, if the risk of developing health problems is known to increase with age, Bayes's theorem allows the risk to an individual of a known age to be assessed more accurately than simply assuming that the individual is typical of the population as a whole.",
    "One of the many applications of Bayes's theorem is Bayesian inference, a particular approach to statistical inference.",
    "When applied, the probabilities involved in Bayes's theorem may have different probability interpretations.",
    "With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence.",
    "Bayesian inference is fundamental to Bayesian statistics.",
    "Bayes's theorem is named after Reverend Thomas Bayes, who first used conditional probability to provide an algorithm (his Proposition 9) that uses evidence to calculate limits on an unknown parameter, published as An Essay towards solving a Problem in the Doctrine of Chances (1763). ",
    "In what he called a scholium, Bayes extended his algorithm to any unknown prior cause.",
    "Independently of Bayes, Pierre-Simon Laplace in 1774, and later in his 1812 Théorie analytique des probabilités, used conditional probability to formulate the relation of an updated posterior probability from a prior probability, given evidence.",
    "Sir Harold Jeffreys put Bayes's algorithm and Laplace’s formulation on an axiomatic basis, writing that Bayes's theorem 'is to the theory of probability what the Pythagorean theorem is to geometry'."
]

2. Guinness Brewery

In [5]:
guinness = [
    "St. James's Gate Brewery (Irish: Grúdlann Gheata Naomh Séamuis) is a brewery founded in 1759 in Dublin, Ireland, by Arthur Guinness.",
    "The company is now a part of Diageo, a British company formed from the merger of Guinness and Grand Metropolitan in 1997.",
    "The main product of the brewery is Guinness Draught.",
    "Originally leased in 1759 to Arthur Guinness at 45pound per year for 9,000 years, the St. James's Gate area has been the home of Guinness ever since.",
    "It became the largest brewery in Ireland in 1838, and the largest in the world by 1886, with an annual output of 1.2 million barrels.",
    "Although no longer the largest brewery in the world, it remains as the largest brewer of stout.",
    "The company has since bought out the originally leased property, and during the 19th and early 20th centuries the brewery owned most of the buildings in the surrounding area, including many streets of housing for brewery employees, and offices associated with the brewery.",
    "The brewery had its own power plant.",
    "There is an attached exhibition on the 250-year-old history of Guinness, called the Guinness Storehouse.",
    "Arthur Guinness started brewing ales in Leixlip, County Kildare, and then from 1759 at the St. James's Gate Brewery in Dublin."
]

3. Google

In [6]:
google = [
    'Google, LLC is an American multinational technology company that specializes in Internet-related services and products, which include online advertising technologies, a search engine, cloud computing, software, and hardware',
    'It is considered one of the Big Four technology companies, alongside Amazon, Apple, and Microsoft',
    'Google was founded in September 1998 by Larry Page and Sergey Brin while they were Ph.D students at Stanford University in California',
    'Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock',
    'They incorporated Google as a California privately held company on September 4, 1998, in California',
    'Google was then reincorporated in Delaware on October 22, 2002',
    'An initial public offering (IPO) took place on August 19, 2004, and Google moved to its headquarters in Mountain View, California, nicknamed the Googleplex',
    'In August 2015, Google announced plans to reorganize its various interests as a conglomerate called Alphabet Inc',
    "Google is Alphabet's leading subsidiary and will continue to be the umbrella company for Alphabet's Internet interests",
    'Sundar Pichai was appointed CEO of Google, replacing Larry Page who became the CEO of Alphabet.'
]

Then extract nouns from each group of sentences. Class `TextBlob` defined in `textblob` package will be used to tag words in the sentence.

In [45]:
def extract_nouns(document):
    return [tag[0] for tag in TextBlob(document).tags if tag[1].startswith('NN')]

nouns = {
    'bayes' : [],
    'guinness' : [],
    'google' : []
}

index = 0
label = ['bayes', 'guinness', 'google']

for category in [bayes, guinness, google]:
    for document in category:
        nouns[label[index]] += extract_nouns(document)
    nouns[label[index]] = set(nouns[label[index]])
    index += 1

As `guinness` and `google` share two words in common, remove them from `google` for convenience.

In [46]:
nouns['guinness'].intersection(nouns['google'])

{'company', 'power'}

In [47]:
nouns['google'] = nouns['google'].difference({'company', 'power'})

Then reconstruct this sets as single list.

In [48]:
words = list(nouns['bayes']) + list(nouns['guinness']) + list(nouns['google'])

## 1. Generate topics

In [50]:
len_bayes = len(nouns['bayes'])
len_guinness = len(nouns['guinness'])
len_google = len(nouns['google'])

Then create hypothetical composition of words from each of these category. Simply assume that occurence frequency of each word in each category is equal.

In [53]:
topic_bayes = [1/len_bayes for i in range(len_bayes)] + [0 for i in range(len_guinness)] + [0 for i in range(len_google)]
topic_guinness = [0 for i in range(len_bayes)] + [1/len_guinness for i in range(len_guinness)] + [0 for i in range(len_google)]
topic_google = [0 for i in range(len_bayes)] + [0 for i in range(len_guinness)] + [1/len_google for i in range(len_google)]

For example they can be thought of as article of Google employee who majored Bayesian Statistics and loves Guiness beer.

Then, define map from word to corresponding index of words and format each documents as coordinates of counts of words.

In [11]:
word2idx = defaultdict(lambda: len(word2idx))
indices = []
for d in range(len(documents)):
    words = extract_nouns(documents[d])
    coordinates = [(d, word2idx[word], count) for word, count in Counter(extract_nouns(documents[d])).items()]
    indices.extend(coordinates)

Using the coordinates, 

In [12]:
corpus = np.zeros((len(documents), len(word2idx)))
for index in indices:
    corpus[index[0], index[1]] = index[2]

In [14]:
model = VanillaLDA(len(documents), len(word2idx), 3)
model.fit(corpus)

ValueError: operands could not be broadcast together with shapes (30,) (3,) 